Since you are operating in a Session-Based Architecture (without a central Metastore/Catalog), you cannot simply go to the "Catalog" tab or run standard SELECT * FROM table commands in the SQL Editor. The tables "physically" exist in S3, but the Databricks UI doesn't know they exist.

To query them, you must register them as Temporary Views inside your notebook. This bridges the gap between your Python credentials and SQL syntax.

In [0]:
ACCESS_KEY = dbutils.secrets.get(scope = "ticker", key = "access_key")
SECRET_KEY = dbutils.secrets.get(scope = "ticker", key = "secret_key")
SESSION_TOKEN = dbutils.secrets.get(scope = "ticker", key = "session_key")

In [0]:
# 1. Setup Credentials (As always)
ACCESS_KEY = dbutils.secrets.get(scope = "ticker", key = "access_key")
SECRET_KEY = dbutils.secrets.get(scope = "ticker", key = "secret_key")
SESSION_TOKEN = dbutils.secrets.get(scope = "ticker", key = "session_key")

# 2. Define Source Paths (S3)
bronze_path = "s3a://mzon-to-databricks-5482/bronze/source=fmp/"
silver_path = "s3a://mzon-to-databricks-5482/silver/income_statement/valid"
quarantine_path = "s3a://mzon-to-databricks-5482/quarantine/company_financials/"

# 3. Define the Secure Reader
def read_secure(path):
    return (spark.read.format("delta")
            .option("fs.s3a.access.key", ACCESS_KEY)
            .option("fs.s3a.secret.key", SECRET_KEY)
            .option("fs.s3a.session.token", SESSION_TOKEN)
            .option("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
            .load(path))

# 4. The "Snapshot" Logic
# We read from S3 (Secure) -> Write to Managed Table (Open)
def refresh_managed_table(table_name, s3_path):
    try:
        print(f"🔄 Syncing {table_name} from S3...")
        df = read_secure(s3_path)
        
        # Overwrite the Managed Table (Saves to DBFS/Local)
        # This makes it queryable via SQL without keys!
        df.write.mode("overwrite").saveAsTable(table_name)
        
        print(f"✅ Success! Table '{table_name}' is now live in Catalog.")
    except Exception as e:
        print(f"⚠️ Could not sync {table_name}. (Data might not exist yet). Error: {e}")

# 5. Execute Sync
refresh_managed_table("bronze_audit", bronze_path)
refresh_managed_table("silver_audit", silver_path)
refresh_managed_table("quarantine_audit", quarantine_path)

A. Check Bronze (Raw History)

In [0]:
%sql
-- 1. How many raw records do we have?
SELECT count(*) as total_bronze_rows FROM v_bronze;

-- 2. Do we have duplicates? (Should be YES if you ran the job twice)
SELECT symbol, date, count(*) as duplicate_count  , eps
FROM v_bronze 
GROUP BY symbol, date ,eps
HAVING count(*) > 1;

B. Check Silver (The "Truth")

In [0]:
%sql
-- 1. Verify Uniqueness (Should return 0 rows)
SELECT symbol, date, count(*) 
FROM v_silver 
GROUP BY symbol, date 
HAVING count(*) > 1;

-- 2. Check Data Types & Values
SELECT symbol, date, revenue, eps, reportedCurrency
FROM v_silver
ORDER BY date DESC
LIMIT 10;

In [0]:
%sql
-- 1. Why are rows failing?
SELECT date, symbol, _failed_cols, revenue, eps
FROM v_quarantine
LIMIT 20;

-- 2. Which error is most common?
SELECT _failed_cols, count(*) as error_count
FROM v_quarantine
GROUP BY _failed_cols;